# Date solved: 17 November 2018

# Problem
https://projecteuler.net/problem=82

The minimal path sum in the 5 by 5 matrix below, by starting in any cell in the left column and finishing in any cell in the right column, and only moving up, down, and right, is indicated in red and bold; the sum is equal to 994.

$$
\begin{pmatrix}
131 & 673 & \color{red}{234} & \color{red}{103} & \color{red}{18}\\
\color{red}{201} & \color{red}{96} & \color{red}{342} & 965 & 150\\
630 & 803 & 746 & 422 & 111\\
537 & 699 & 497 & 121 & 956\\
805 & 732 & 524 & 37 & 331
\end{pmatrix}
$$

Find the minimal path sum, in matrix.txt (right click and "Save Link/Target As..."), a 31K text file containing a 80 by 80 matrix, from the left column to the right column.

# Solution
This problem is very similar to Problem 81: we can solve this problem by applying the shortest path algorithm to a directed graph defined on the matrix. But there are a few differences: 

#### Source and target nodes
In Problem 81, we had only one source node and one target node. However, in this problem, we are allowed to start at any position in the left column and we have to end at some position in the right column. This means that we have $n$ source and target nodes, where $n$ is the number of rows in the matrix.

#### Path movement
In Problem 81, we were only allowed to move to the right and downwards, but in this problem we are allowed to move in all directions except to the left. This means that we need to add 1 more edge to each node (if possible), namely an edge that corresponds to the upwards direction.

## Construction of the directed graph
Let $G$ be a directed graph. The number of nodes is equal to the number of elements in the matrix + the number of source nodes (it will become clear why this is necessary). Using the graph above, this means that we need 25 + 5 nodes to construct the corresponding directed graph. We denote the source nodes as tuples of $$\{(s, 0), (s, 1), \dots, (s, 4)\},$$ where $(s, i)$ denotes the $i+1$-th source node. 

Moreover, we denote the other nodes with tuples $$\{(0,0), (0,1), \dots, (4,3), (4,4)\},$$ where each tuple $(i,j)$ corresponds to the $i+1,j+1$-th element of the matrix. Mathematically speaking, the top-left element in the matrix is $a_{11}$, i.e. it has subscript $(1,1)$, but since Python starts indexing with 0, this indexing is more convenient. Hence, the node $(0,0)$ corresponds to the matrix element $a_{11}$. 

Next, we construct the edges of $G$. Note that we only construct directed edges in this example. Lets start with the source nodes. We add one edge from each source node $(s, i)$ to the node $(i,0)$ with length equal to the corresponding matrix element. For example, the edge from $(s, 0)$ to $(0,0)$ has length 131.

Furthermore, observe that we can move to the right, down *and up* in the matrix, so we need to construct an edge from element to its right neighbour, its bottom neighbour and its top neighbour. Hence, for each node $(i, j)$, we construct:

* an edge from $(i, j)$ to $(i, j+1)$ (right neighbour);
* an edge from $(i,j)$ to $(i+1,j)$ (bottom neighbour); 
* and an edge from $(i,j)$ to $(i-1,j)$ (top neighbour). 

The length of the edge ($(i,j)$, $(k,l)$) corresponds to the matrix value of $(k,l)$. 

Note that we only construct an edge if the neighbour exists (so for node $(4,4)$, there exists no right nor bottom neighbour and hence we do not add any edges.)

This completes the graph construction. 

In [3]:
import networkx as nx
import numpy as np

mat = np.array([x.split(',') 
                for x in [l.rstrip() 
                          for l in open("pe82.txt").readlines()]], 
               dtype='int32')

In [6]:
def mat_to_DiGraph(M):
    """Constructs a directed graph (for Problem 82) from a numpy matrix."""
    G = nx.DiGraph()
    n = M.shape[0] # number of rows
    m = M.shape[1] # number of cols
    
    # Construct an edge from each source node to the matrix nodes
    for i in range(0, n):
        G.add_edge(('s',i),(i,0),weight=M[i,0])
    
    # Construct the edges for other nodes
    for i in range(0, n):
        for j in range(0, m):
            # Add right neighbour
            if j < m - 1:
                G.add_weighted_edges_from([((i,j),(i,j+1),M[i,j+1])])            
            
            # Add bottom neighbour
            if i < n - 1:
                G.add_weighted_edges_from([((i,j),(i+1,j),M[i+1,j])])
            
            # Add top neighbour
            if i > 0:
                G.add_weighted_edges_from([((i,j),(i-1,j),M[i-1,j])])
    
    return(G)

def minPathSum(M):
    """Finds the minimum path sum from Problem 82."""
    G = mat_to_DiGraph(M)
    n, m = [M.shape[0],M.shape[1]]
    minPath = M[0,:].sum()
    for i in range(n):
        for j in range(m):
            # Calculate the shortest path from every source node to every target node
            path = nx.dijkstra_path_length(G, ('s', i), (j,m-1))
            if path < minPath:
                minPath = path
    
    return(minPath)

print(minPathSum(mat))

260324


# Conclusion
Though this solution works, its computationally very expensive to perform. Dijkstra's shortest path algorithm is $O(|V|^{2})$, but we apply the shortest path for each possible combination of source and target, i.e. $|V|^{2}$ times. The total computation time is then $O(|V|^{4})$, which is clearly not that good. 